In [28]:
import collections
import requests, sys
import re
import os
from ete3 import Tree 
os.chdir('/Users/otoolea2/Documents/phd')
os.getcwd()

'/Users/otoolea2/Documents/phd'

# Defining the macaque and gibbon singleton genes using blast

In [132]:
translator = {} 
files = ['mac_gid_tid_pid.txt', 'gib_gid_tid_pid.txt']
for gp in files:
    with open("gid_tid_pid/" + gp,"r") as f:
        for l in f:
            l = l.rstrip('\n')
            tokens = l.split(',')
            if tokens[2] not in translator and tokens[2] != '':
                translator[tokens[2]] = tokens[0]
print len(translator)

56372


In [133]:
genes_with_hits = collections.defaultdict(list)  
uniqued = collections.defaultdict(list)
with open("ensembl_protein_seq_for_blast/blast_results/gib_v_gib_blast","r") as f:
    for l in f:
        l = l.rstrip('\n')
        tokens = l.split()
        if tokens[1].startswith('ENSNLEP'):
            if tokens[1] in translator and tokens[0] in translator:
                if float(tokens[10]) < 0.1 and translator[tokens[1]] != translator[tokens[0]]:
                    genes_with_hits[translator[tokens[0]]].append(translator[tokens[1]])
for query in genes_with_hits:
    uniqued[query].append(list(set(genes_with_hits[query])))

print("have added genes to the dictionary with eval < 0.1. len of dict is: " + str(len(uniqued)))

have added genes to the dictionary with eval < 0.1. len of dict is: 15614


In [134]:
gs = []
with open("gid_tid_pid/gib_gid_tid_pid.txt","r") as f:  
    for l in f:
        l = l.rstrip('\n')
        tokens = l.split(',')
        if (tokens[0] in uniqued.keys()):
            pass
        else: 
            gs.append(tokens[0])
            
f = open("rbh/gibbon_singletons.txt","w")            
gs = list(set(gs))
for item in gs:
    f.write(item + '\n')
f.close()
print('number of gibbon singleton type hits is ' + str(len(gs)))

number of gibbon singleton type hits is 2961


In [135]:
genes_with_hits = collections.defaultdict(list)  
uniqued = collections.defaultdict(list)
with open("ensembl_protein_seq_for_blast/blast_results/mac_v_mac_blast","r")as f:
    for l in f:
        l = l.rstrip('\n')
        tokens = l.split()
        if tokens[1] in translator and tokens[0] in translator:
            if float(tokens[10]) < 0.1 and translator[tokens[1]] != translator[tokens[0]]:
                genes_with_hits[translator[tokens[0]]].append(translator[tokens[1]])
for query in genes_with_hits:
    uniqued[query].append(list(set(genes_with_hits[query])))
    
print("have added genes to the dictionary with eval < 0.1. len of dict is: " + str(len(uniqued)))

have added genes to the dictionary with eval < 0.1. len of dict is: 19106


In [136]:
ms = []
with open("gid_tid_pid/mac_gid_tid_pid.txt","r") as f:  
    for l in f:
        l = l.rstrip('\n')
        tokens = l.split(',')
        if (tokens[0] in uniqued.keys()):
            pass
        else: 
            ms.append(tokens[0])

f = open("rbh/macaque_singletons.txt","w")            
ms = list(set(ms))             
for item in ms:
    f.write(item + '\n')
f.close()
print('number of macaque singleton type hits is ' + str(len(ms)))

number of macaque singleton type hits is 2799


In [137]:
longest_translator = {}
c1 = 0
c2 = 0 
c3 = 0
with open('duplicability/longest_mac_and_gib_protein_gene_id.txt','r') as f:
    for l in f:
        l = l.rstrip('\n')
        tokens = l.split(',')
        longest_translator[tokens[1]]=tokens[0]
        c1 += 1
        
f_1 = open('rbh/gibbon_singletons_pid.txt','w')
with open('rbh/gibbon_singletons.txt') as f:
    for l in f:
        l = l.rstrip('\n')
        f_1.write(str(longest_translator[l]) + '\n')
        c2 += 1
f_1.close()
f_1 = open('rbh/macaque_singletons_pid.txt','w')
with open('rbh/macaque_singletons.txt') as f:
    for l in f:
        l = l.rstrip('\n')
        f_1.write(str(longest_translator[l]) + '\n') 
        c3 += 1
f_1.close()

print c1, c2, c3

40480 2961 2799


In [138]:
singleton_gib = []
with open('rbh/gibbon_singletons.txt','r') as f:
    for l in f:
        l = l.rstrip('\n')
        singleton_gib.append(l)
print len(singleton_gib)
singleton_mac = []
with open('rbh/macaque_singletons.txt','r') as f:
    for l in f:
        l = l.rstrip('\n')
        singleton_mac.append(l)
print len(singleton_mac)

2961
2799


# Getting and sorting the gene trees from the gib singleton list

In [139]:
import requests, sys
import re

server = "http://rest.ensembl.org"
ext = "/genetree/member/id/" 

num_r = 0
not_okay = []
long_k = 0
both_more1 = 0
both_more2 = 0
mac_more = 0
gib_more = 0
c_sing = 0
mac_gene_loss = []


for gene_id in singleton_gib:
    num_r += 1
    if num_r % 500 == 0:
        print ('progressing')
    r = requests.get(server+ext+gene_id+"?", headers={ "Content-Type" : "text/x-nh"})
    #print(r.url)
    if not r.ok:
        not_okay.append(gene_id)
        continue
        r.raise_for_status()
        sys.exit()
        
    k = re.findall("(NLEP|MMUP)", r.text)
    if len(k) > 2:
        long_k +=1

        if k.count('MMUP') > 1 and k.count('NLEP') > 1:
            both_more1 += 1
            j = re.search("filename=(\w+\.nh)", r.headers['content-disposition'])
            with open('gene_trees/both_more/' + j.groups()[0],'w') as f:
                f.write(r.text)
                both_more2 += 1
        elif k.count('MMUP') > 1 and k.count('NLEP') == 1:
            mac_more += 1
            j = re.search("filename=(\w+\.nh)", r.headers['content-disposition'])
            with open('gene_trees/mac_more/' + j.groups()[0],'w') as f:
                f.write(r.text)
        elif k.count('MMUP') == 1 and k.count('NLEP') > 1:
            gib_more += 1
            j = re.search("filename=(\w+\.nh)", r.headers['content-disposition'])
            with open('gene_trees/gib_more/' + j.groups()[0],'w') as f:
                f.write(r.text)
            
    elif k.count('MMUP') < 1 and k.count('NLEP') == 1:
        mac_gene_loss.append(gene_id)
        
    elif k.count('MMUP') == 1 and k.count('NLEP') == 1:
        c_sing += 1
        j = re.search("filename=(\w+\.nh)", r.headers['content-disposition'])
        with open('gene_trees/gibbon_singleton/' + j.groups()[0],'w') as f:
                f.write(r.text)
                    
print num_r
print len(not_okay)
print long_k, both_more1, both_more2
print mac_more, gib_more
print len(mac_gene_loss)
print c_sing


progressing
progressing
progressing
progressing
progressing
2961
130
343 52 52
247 44
562
1926


# Finding singletons for trees with high acn

In [140]:
import os 

indir = 'gene_trees/both_more'
c = 0
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/both_more/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            print t2
print c


               /-ENSP00000364204
            /-|
         /-|   \-ENSPTRP00000000500
        |  |
      /-|   \-ENSGGOP00000014209
     |  |
   /-|   \-ENSPPYP00000002052
  |  |
  |  |   /-ENSNLEP00000024309
--|   \-|
  |      \-ENSNLEP00000009658
  |
  |   /-ENSMMUP00000000169
   \-|
      \-ENSMMUP00000011580

                  /-ENSGGOP00000008665
               /-|
            /-|   \-ENSPTRP00000016916
           |  |
         /-|   \-ENSP00000237014
        |  |
      /-|   \-ENSPPYP00000010206
     |  |
   /-|   \-ENSNLEP00000007850
  |  |
  |   \-ENSMMUP00000021922
--|
  |      /-ENSNLEP00000019491
  |   /-|
  |  |   \-ENSPTRP00000060710
   \-|
     |   /-ENSMMUP00000032255
      \-|
         \-ENSGGOP00000018525

         /-ENSGGOP00000020666
      /-|
     |   \-ENSMMUP00000037364
     |
     |            /-ENSGGOP00000017141
   /-|         /-|
  |  |      /-|   \-ENSGGOP00000026169
  |  |     |  |
  |  |   /-|   \-ENSP00000386531
  |  |  |  |
  |   \-|   \-ENSPPYP0000001495

In [141]:
#print t2
c = 0
for node in t2.traverse():
    if len(node) == 6:
        print node.name
        n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
        print n
        print n.count('MMUP')
        print n.count('NLEP')
        print n.count('PPYP')
        print n.count('GGOP')
        print n.count('PTRP')
        print n.count('ENSP0')
        if n.count('MMUP') == 1 and n.count('NLEP') == 1 and n.count('PPYP') == 1:
            if n.count('GGOP') == 1 and n.count('PTRP') == 1 and n.count('ENSP0') == 1:
                print('singleton')
                genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                genes = sorted(genes0)
                node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
                c +=1
                
print c
        
        
    



['ENSP0', 'PTRP', 'GGOP', 'PPYP', 'NLEP', 'MMUP']
1
1
1
1
1
1
singleton

['PTRP', 'ENSP0', 'GGOP', 'PPYP', 'NLEP', 'MMUP']
1
1
1
1
1
1
singleton
2


In [142]:
indir = 'gene_trees/both_more'
c = 0
sing = 0
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/both_more/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            for node in t2.traverse():
                if len(node) == 6:
                    n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
                    if n.count('MMUP') == 1 and n.count('NLEP') == 1 and n.count('PPYP') == 1:
                        if n.count('GGOP') == 1 and n.count('PTRP') == 1 and n.count('ENSP0') == 1:
                            if node.children[1].name.startswith('ENSMMUP'): 
                                if node.children[0].children[1].name.startswith('ENSNLEP') or node.children[0].children[0].name.startswith('ENSNLEP'):
                                    sing +=1
                                    genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                    genes = sorted(genes0)
                                    node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
                            elif node.children[0].name.startswith('ENSMMUP'):
                                if node.children[1].children[1].name.startswith('ENSNLEP') or node.children[1].children[0].name.startswith('ENSNLEP'):
                                    sing +=1
                                    genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                    genes = sorted(genes0)
                                    node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
print c, sing

44 67


In [143]:
indir = 'gene_trees/gib_more'
c = 0
sing = 0
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/gib_more/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            for node in t2.traverse():
                if len(node) == 6:
                    n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
                    if n.count('MMUP') == 1 and n.count('NLEP') == 1 and n.count('PPYP') == 1:
                        if n.count('GGOP') == 1 and n.count('PTRP') == 1 and n.count('ENSP0') == 1:
                            if node.children[1].name.startswith('ENSMMUP'): 
                                if node.children[0].children[1].name.startswith('ENSNLEP') or node.children[0].children[0].name.startswith('ENSNLEP'):
                                    sing +=1
                                    genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                    genes = sorted(genes0)
                                    node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
                            elif node.children[0].name.startswith('ENSMMUP'):
                                if node.children[1].children[1].name.startswith('ENSNLEP') or node.children[1].children[0].name.startswith('ENSNLEP'):
                                    sing +=1
                                    genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                    genes = sorted(genes0)
                                    node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
        
print c, sing

30 8


In [145]:
indir = 'gene_trees/mac_more'
c = 0
sing = 0
for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/mac_more/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            for node in t2.traverse():
                if len(node) == 6:
                    n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
                    if n.count('MMUP') == 1 and n.count('NLEP') == 1 and n.count('PPYP') == 1:
                        if n.count('GGOP') == 1 and n.count('PTRP') == 1 and n.count('ENSP0') == 1:
                            if node.children[1].name.startswith('ENSMMUP'): 
                                if node.children[0].children[1].name.startswith('ENSNLEP') or node.children[0].children[0].name.startswith('ENSNLEP'):
                                    sing +=1
                                    genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                    genes = sorted(genes0)
                                    node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
                            elif node.children[0].name.startswith('ENSMMUP'):
                                if node.children[1].children[1].name.startswith('ENSNLEP') or node.children[1].children[0].name.startswith('ENSNLEP'):
                                    sing +=1
                                    genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                    genes = sorted(genes0)
                                    node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
            
        
print c, sing

247 76


# Finding multiple hits for gene trees with high acn

In [146]:
from ete3 import Tree
indir = 'gene_trees/mac_more'
c = 0
mult = 0

for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/mac_more/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            for node in t2.traverse():
                n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
                if n.count('MMUP') == 1 and n.count('NLEP') == 1:
                    if n.count('PPYP') > 1 or n.count('GGOP') > 1 or n.count('PTRP') > 1 or n.count('ENSP0') > 1:
                        if node.children[1].name.startswith('ENSMMUP') or node.children[0].name.startswith('ENSMMUP'):
                            if node.children[0].children[1].name.startswith('ENSNLEP') or node.children[0].children[0].name.startswith('ENSNLEP'):
                                mult += 1
                                print node
                                genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                genes = sorted(genes0)
                                node.write(format=1, outfile='gene_trees/pruned_trees/mult_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
print c, mult


               /-ENSP00000358233
            /-|
           |   \-ENSP00000319673
         /-|
        |  |   /-ENSPTRP00000002068
      /-|   \-|
     |  |      \-ENSPTRP00000060671
   /-|  |
  |  |   \-ENSPPYP00000001093
--|  |
  |   \-ENSNLEP00000006783
  |
   \-ENSMMUP00000025803

            /-ENSPPYP00000023458
         /-|
        |   \-ENSPPYP00000020452
      /-|
     |  |      /-ENSPTRP00000032761
     |  |   /-|
   /-|   \-|   \-ENSGGOP00000000612
  |  |     |
  |  |      \-ENSP00000258774
--|  |
  |   \-ENSNLEP00000005464
  |
   \-ENSMMUP00000009228

                  /-ENSP00000352257
               /-|
            /-|   \-ENSPTRP00000043206
           |  |
         /-|   \-ENSGGOP00000015496
        |  |
      /-|   \-ENSPTRP00000056956
     |  |
   /-|   \-ENSPPYP00000013253
  |  |
--|   \-ENSNLEP00000018935
  |
   \-ENSMMUP00000036966

                  /-ENSPTRP00000057814
               /-|
            /-|   \-ENSP00000476672
           |  |
         /-|   \-ENSGGOP0

In [147]:
indir = 'gene_trees/gib_more'
c = 0
mult = 0

for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/gib_more/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            for node in t2.traverse():
                n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
                if n.count('MMUP') == 1 and n.count('NLEP') == 1:
                    if n.count('PPYP') > 1 or n.count('GGOP') > 1 or n.count('PTRP') > 1 or n.count('ENSP0') > 1:
                        if node.children[1].name.startswith('ENSMMUP') or node.children[0].name.startswith('ENSMMUP'):
                            if node.children[0].children[1].name.startswith('ENSNLEP') or node.children[0].children[0].name.startswith('ENSNLEP'):
                                mult += 1
                                print node
                                genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                genes = sorted(genes0)
                                node.write(format=1, outfile='gene_trees/pruned_trees/mult_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
print c, mult
        
    

30 0


In [148]:
indir = 'gene_trees/both_more'
c = 0
mult = 0

for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/both_more/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            for node in t2.traverse():
                n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
                if n.count('MMUP') == 1 and n.count('NLEP') == 1:
                    if n.count('PPYP') > 1 or n.count('GGOP') > 1 or n.count('PTRP') > 1 or n.count('ENSP0') > 1:
                        if node.children[1].name.startswith('ENSMMUP') or node.children[0].name.startswith('ENSMMUP'):
                            if node.children[0].children[1].name.startswith('ENSNLEP') or node.children[0].children[0].name.startswith('ENSNLEP'):
                                mult += 1
                                print node
                                genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                genes = sorted(genes0)
                                node.write(format=1, outfile='gene_trees/pruned_trees/mult_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
print c, mult


                  /-ENSGGOP00000021985
               /-|
            /-|   \-ENSGGOP00000000449
           |  |
         /-|   \-ENSPTRP00000031410
        |  |
      /-|   \-ENSP00000358739
     |  |
   /-|   \-ENSPPYP00000018813
  |  |
--|   \-ENSNLEP00000015382
  |
   \-ENSMMUP00000016526

               /-ENSP00000401513
            /-|
           |   \-ENSPTRP00000011955
         /-|
        |  |   /-ENSGGOP00000004567
      /-|   \-|
     |  |      \-ENSGGOP00000017826
   /-|  |
  |  |   \-ENSPPYP00000007275
--|  |
  |   \-ENSNLEP00000006464
  |
   \-ENSMMUP00000028758

                  /-ENSPTRP00000015115
               /-|
            /-|   \-ENSGGOP00000009402
           |  |
           |   \-ENSP00000379025
           |
         /-|         /-ENSP00000402644
        |  |      /-|
        |  |   /-|   \-ENSGGOP00000022826
        |  |  |  |
      /-|   \-|   \-ENSP00000379242
     |  |     |
     |  |      \-ENSPTRP00000039166
   /-|  |
  |  |   \-ENSPPYP00000009002
--|  |

# Filtering through the gibbon singleton gene trees with k==2

In [149]:
indir = 'gene_trees/gibbon_singleton'
c = 0
c1 = 0
sing = 0

for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/gibbon_singleton/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            for node in t2.traverse():
                if len(node) == 6:
                    n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
                    if n.count('MMUP') == 1 and n.count('NLEP') == 1 and n.count('PPYP') == 1:
                        if n.count('GGOP') == 1 and n.count('PTRP') == 1 and n.count('ENSP0') == 1:
                            c1+=1
                            if node.children[1].name.startswith('ENSMMUP'): 
                                if node.children[0].children[1].name.startswith('ENSNLEP') or node.children[0].children[0].name.startswith('ENSNLEP'):
                                    sing +=1
                                    genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                    genes = sorted(genes0)
                                    node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
                            elif node.children[0].name.startswith('ENSMMUP'):
                                if node.children[1].children[1].name.startswith('ENSNLEP') or node.children[1].children[0].name.startswith('ENSNLEP'):
                                    sing +=1
                                    genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                    genes = sorted(genes0)
                                    node.write(format=1, outfile='gene_trees/pruned_trees/singleton_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
print c, c1, sing

1926 1507 1327


In [150]:
indir = 'gene_trees/gibbon_singleton'
c = 0
c1 = 0
mult = 0

for r, d, f in os.walk(indir):
    for my_file in f:
        if my_file.endswith('.nh'):
            c+=1
            t = Tree('gene_trees/gibbon_singleton/' + my_file, format=0)
            #print t
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            t2 = Tree(t.write(t.prune(sp)))
            for node in t2.traverse():
 
                n = re.findall("(ENSP\d|NLEP|PTRP|GGOP|MMUP|PPYP)",str(node))
                if n.count('MMUP') == 1 and n.count('NLEP') == 1:
                    if n.count('GGOP') > 1 or n.count('PTRP') > 1 or n.count('ENSP0') > 1 or n.count('PPYP') > 1:
                        c1 += 1
                        if node.children[1].name.startswith('ENSMMUP'): 
                            if node.children[0].children[1].name.startswith('ENSNLEP') or node.children[0].children[0].name.startswith('ENSNLEP'):
                                mult +=1
                                genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                genes = sorted(genes0)
                                node.write(format=1, outfile='gene_trees/pruned_trees/mult_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
                        elif node.children[0].name.startswith('ENSMMUP'):
                            if node.children[1].children[1].name.startswith('ENSNLEP') or node.children[1].children[0].name.startswith('ENSNLEP'):
                                mult +=1
                                genes0 = re.findall("(ENSNLEP\d+|ENSMMUP\d+)",str(node))
                                genes = sorted(genes0)
                                node.write(format=1, outfile='gene_trees/pruned_trees/mult_high_acn/'+my_file.rstrip('.nh')+'_'+genes[0]+'_'+genes[1]+'.nw')
print c, c1, mult

1926 223 107


In [151]:
indir = 'gene_trees/pruned_trees/mult_high_acn'
c=0
for r, d, f in os.walk(indir):
    for l in f:
        if l.endswith('.nw'):
            c+=1
print c

125


In [152]:
indir = 'gene_trees/pruned_trees/singleton_high_acn'
c=0
for r, d, f in os.walk(indir):
    for l in f:
        if l.endswith('.nw'):
            c+=1
print c

1478


# Gene Loss vs Dup Alignments

In [161]:
import ete3
import requests, sys
import re
import json
import os
import collections

server = "http://rest.ensembl.org"
ext = "/sequence/id"
headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

indir = 'gene_trees/pruned_trees/mult_high_acn'
for r, d, f in os.walk(indir):
    for l in f:
        if l.endswith('.nw'):
            
            t = Tree(r + '/' + l, format=0)
            sp = re.findall("(ENSP\d+|ENSNLEP\d+|ENSPTRP\d+|ENSGGOP\d+|ENSMMUP\d+|ENSPPYP\d+)",str(t))
            orth = []
            for item in sp:
                if item.startswith('ENSMMUP'):
                    pass
                elif item.startswith('ENSNLEP'):
                    gib = item
                    orth.append(gib)
                else:
                    orth.append(item)
            
            if not os.path.exists('duplicability/dup_v_loss_again/'+gib):
                os.makedirs('duplicability/dup_v_loss_again/'+gib)
            
            req = requests.post(server+ext, headers=headers, data='{ "ids" : ' + json.dumps(orth) + ' }') 
            
            if not req.ok:
                continue
                req.raise_for_status()
                sys.exit()
            decoded = req.json()
            
            with open('duplicability/dup_v_loss_again/' + gib +'/my_tester_file.txt','w') as f_t:
                f_t.write('this is an identifier file')
                
            for result in decoded:
                f_2 = open('duplicability/dup_v_loss_again/' + gib + '/'+ result['id'] + '_protein_seq.fa','w')
                f_2.write('>' + result['id'] + '\n' + result['seq'] + '\n')
                f_2.close()    

In [159]:
print sp

print req

print decoded

['ENSPPYP00000022381', 'ENSPPYP00000020555', 'ENSPTRP00000054328', 'ENSGGOP00000003628', 'ENSP00000382951', 'ENSP00000392568', 'ENSPTRP00000059204', 'ENSNLEP00000018348', 'ENSMMUP00000032228']
<Response [200]>
[{u'molecule': u'protein', u'id': u'ENSPPYP00000022381', u'seq': u'MKLHSLISVLLLFVTLIPKGKTGVIPGQKQCIALKGVCRDKLCSTLDDTIGICNEEKKCCRRWWIFEPYPTPVPKGKSP', u'desc': None}, {u'molecule': u'protein', u'id': u'ENSPPYP00000020555', u'seq': u'MKLHSLISVLLLFVTLIPKGKTGVIPGQKQCIALKGVCRDKLCSTLDDTIGICNEEKKCCRRWWIFEPYPTPVPEGKSP', u'desc': None}, {u'molecule': u'protein', u'id': u'ENSPTRP00000054328', u'seq': u'MKLHSLISVLLLFVTLIPKGKTGVIPGQKQCIALKGVCRDKLCSTLDDTIGICNEGKKCCRRWWILEPYPTPVPKGKSP', u'desc': None}, {u'molecule': u'protein', u'id': u'ENSGGOP00000003628', u'seq': u'IKLHSLISVLLLFVSLIPKGKTGVIPGQKQIALKGVWRDKLCSTLDDTIGICNEGKKCCKKWWILKPYPTPVPKGKSP', u'desc': None}, {u'molecule': u'protein', u'id': u'ENSP00000382951', u'seq': u'MKLHSLISVLLLFVTLIPKGKTGVIPGQKQCIALKGVCRDKLCSTLDDTIGICNEGKKCCRRWWILEPYPT

# Can't run needle here, need to use server. #bullshit

In [166]:
import os
import subprocess

indir = 'dup_v_loss_again/'
g = os.walk(indir)
for r, d, f in g:
    if len(f) > 3:
        if os.path.exists(r+'/'+'my_tester_file.txt'):
            beta_files = []
            for my_file in f:  
                if my_file.startswith('ENSNLEP'):
                    gib_file = r + '/' + my_file
                    #print gib_file
                elif my_file.startswith('ENS'):
                    beta_files.append(r + '/' + my_file)
                    #print my_file
            print len(beta_files)
            for b_file in beta_files:
                subprocess.call(['needle','-asequence',gib_file,'-bsequence',b_file,'-gapopen','10.0','-gapextend','0.5','-outfile', b_file.rstrip('.fa')+ '_needle.fa','-aformat','score'])

5


OSError: [Errno 2] No such file or directory

In [164]:
print b_file

duplicability/dup_v_loss_again/ENSNLEP00000000002/ENSGGOP00000001725_protein_seq.fa


In [165]:
print gib_file

duplicability/dup_v_loss_again/ENSNLEP00000000002/ENSNLEP00000000002_protein_seq.fa


In [168]:
subprocess.call(['needle','-asequence','duplicability/dup_v_loss_again/ENSNLEP00000000002/ENSNLEP00000000002_protein_seq.fa','-bsequence','duplicability/dup_v_loss_again/ENSNLEP00000000002/ENSGGOP00000001725_protein_seq.fa','-gapopen','10.0','-gapextend','0.5','-outfile', 'duplicability/dup_v_loss_again/ENSNLEP00000000002/ENSGGOP00000001725_protein_seq_needle.fa','-aformat','score'])

OSError: [Errno 2] No such file or directory

In [ ]:
import os
import subprocess
 
rootDir = './'
g = os.walk(rootDir)

for r, d, f in g:
    if os.path.exists(r+'/'+'my_tester_file.txt'):
        print 'it exists'
        beta_files = []
        for my_file in f:  
            if my_file.startswith('ENSNLEP'):
                gib_file = r + '/' + my_file
                print gib_file
            elif my_file.startswith('ENS'):
                beta_files.append(r + '/' + my_file)
                print my_file
        print len(beta_files)
        for b_file in beta_files:
            subprocess.call(['needle','-asequence',gib_file,'-bsequence',b_file,'-gapopen','10.0','-gapextend','0.5','-outfile', b_file.rstrip('.fa')+ '_needle.fa','-aformat','score'])


                
                

In [ ]:
import os
import collections
import subprocess
 
indir = 'dup_v_loss_again'
g = os.walk(indir)
dups = collections.defaultdict(list)
directories = 0
for r, d, f in g:
  
    if os.path.exists(r+'/'+'my_tester_file.txt'):
        #print('it exists')
        
        if len(f) > 3:
            directories += 1
            scores = []
            for my_file in f:  
                if my_file.endswith('needle.fa'):
                    with open(r+'/'+my_file,'r') as f_1:
                        for l in f_1:
                            if l.startswith('E'):
                                l = l.rstrip('\n')
                                tokens = l.split()
                                scores.append((tokens[1],tokens[3].lstrip('(').rstrip(')')))
            sorted_scores = sorted(scores)
            sp = collections.defaultdict(list)
            sptop = collections.defaultdict(list)

            for item in sorted_scores:
                if item[0].startswith('ENSGGOP'):
                    sp['g'].append(item)
                if item[0].startswith('ENSPPYP'):
                    sp['p'].append(item)
                if item[0].startswith('ENSPTRP'):         
                    sp['c'].append(item)
                if item[0].startswith('ENSP0'):
                    sp['h'].append(item)
            #print sp

            for k in sp:
                if len(sp[k]) > 2:
                    top_scores = sorted(sp[k], key = lambda x : x[1])[:2]
                    sptop[k] = top_scores
                elif len(sp[k]) == 2:
                    sptop[k] = sp[k]
                elif len(sp[k]) < 2:
                    pass
            #print sptop
            for k in sptop:
                print sptop[k][0][0], sptop[k][1][0]
                subprocess.call(['needle','-asequence',r + '/' + str(sptop[k][0][0]) + '_protein_seq.fa','-bsequence',r + '/' + str(sptop[k][1][0]) + '_protein_seq.fa','-gapopen','10.0','-gapextend','0.5','-outfile', r + '/' +'multi_hit_' + str(sptop[k][0][0]) + '_' + str(sptop[k][1][0]) +'_needle_sister.fa','-aformat3','score'])
                with open( r + '/' +'multi_hit_' + str(sptop[k][0][0]) + '_' + str(sptop[k][1][0]) +'_needle_sister.fa','r') as f_a:
                    for l in f_a:
                        l = l.rstrip('\n')
                        if l.startswith('E'):
                            tokens = l.split(' ')
                            dup_score = float(tokens[3].rstrip(')').lstrip('('))
                            if dup_score > float(sptop[k][0][1]) and dup_score > float(sptop[k][1][1]):
                                dups[r].append((sptop[k][0][0], sptop[k][1][0]))
print len(dups)
with open('duplicate_genes.txt','w') as f_d:
    for k in dups:
        f_d.write(str(k) + ',' + str(dups[k]) + '\n')
#print directories
for k in dups:
    print k, dups[k]                                
            
                
                


In [183]:
scores = [('g1', 12),('c1', 34), ('g2', 66), ('g3', 12), ('c2', 299), ('g2', 3), ('c3', 0), ('g4', 1)]
sp = collections.defaultdict(list)
sptop = {}
for item in scores:
    if item[0].startswith('g'):
        sp['g'].append(item)
    elif item[0].startswith('c'):
        sp['c'].append(item)
print sp
for k in sp:
    if len(sp[k]) > 2:
        print sp[k]
        top_scores = sorted(sp[k], key=lambda x : x[1], reverse=True)[:2]
        print top_scores
        sptop[k] = top_scores
print sptop

defaultdict(<type 'list'>, {'c': [('c1', 34), ('c2', 299), ('c3', 0)], 'g': [('g1', 12), ('g2', 66), ('g3', 12), ('g2', 3), ('g4', 1)]})
[('c1', 34), ('c2', 299), ('c3', 0)]
[('c2', 299), ('c1', 34)]
[('g1', 12), ('g2', 66), ('g3', 12), ('g2', 3), ('g4', 1)]
[('g2', 66), ('g1', 12)]
{'c': [('c2', 299), ('c1', 34)], 'g': [('g2', 66), ('g1', 12)]}


In [185]:
for k in sptop:
    print sptop[k][0][0], sptop[k][1][0]
    print sptop[k][0][1], sptop[k][1][1]


c2 c1
299 34
g2 g1
66 12


In [200]:
import os
import shutil 

dup_gib = []
with open('duplicability/dup_v_loss_again/duplicate_genes.txt','r') as f:
    for l in f:
        l = l.rstrip('\n')
        tokens = l.split(',')
        dup_gib.append(tokens[0])
        
print len(dup_gib)

indir = 'duplicability/msa_files/mult'
g = os.walk(indir)
c = 0
for r, d, f in g:
    t = r.split('/')
    genes = t[-1].split('_')
    for gene in genes:
        if gene.startswith('ENSNLEP'):
            if gene in dup_gib:
                print gene
                if not os.path.exists('duplicability/msa_files/dup/'+t[-1]):
                    os.makedirs('duplicability/msa_files/dup/'+t[-1])
                shutil.copy(r + '/' + 'protein_seqs.fa', 'duplicability/msa_files/dup/'+t[-1]+'/protein_seqs.fa')
                shutil.copy(r + '/' + 'nuc_seqs.fa', 'duplicability/msa_files/dup/'+t[-1]+'/nuc_seqs.fa')    
                c+=1
        
print c    
  

45
ENSNLEP00000010553
ENSNLEP00000019010
ENSNLEP00000006783
ENSNLEP00000010545
ENSNLEP00000012510
ENSNLEP00000018935
ENSNLEP00000000734
ENSNLEP00000000002
ENSNLEP00000023813
ENSNLEP00000018273
ENSNLEP00000009688
ENSNLEP00000018687
ENSNLEP00000002458
ENSNLEP00000015765
ENSNLEP00000015966
ENSNLEP00000001848
ENSNLEP00000007580
ENSNLEP00000015382
ENSNLEP00000009742
ENSNLEP00000023967
ENSNLEP00000009202
ENSNLEP00000009820
ENSNLEP00000020398
ENSNLEP00000020947
ENSNLEP00000021067
ENSNLEP00000016480
ENSNLEP00000005138
ENSNLEP00000018463
ENSNLEP00000015700
ENSNLEP00000002116
ENSNLEP00000001139
ENSNLEP00000022206
ENSNLEP00000014508
ENSNLEP00000022977
ENSNLEP00000017420
ENSNLEP00000022547
ENSNLEP00000015115
ENSNLEP00000012940
ENSNLEP00000018932
ENSNLEP00000004361
ENSNLEP00000014347
ENSNLEP00000023820
ENSNLEP00000010265
ENSNLEP00000016369
ENSNLEP00000018348
45
